In [1]:
import sys
import os
sys.path.insert(0, os.path.join(os.getcwd(), '../core'))
import kaggle_support as kgs
import importlib
import matplotlib.pyplot as plt
import numpy as np
import cupy as cp
from dataclasses import dataclass, field, fields
import pack_cuda
import pack_vis
import pack_cost
import copy
import time
import pack_ga
from IPython.display import HTML, display, clear_output
pack_cuda.USE_FLOAT32 = True
pack_cuda._ensure_initialized()


vast
vast
stop final relax at some point


In [ ]:
importlib.reload(pack_ga)
fastMode = False
runner = pack_ga.GA()
runner.N_trees_to_do = np.array([40])
if fastMode:
    runner.initializer.jiggler.duration_compact /= 10
    runner.initializer.jiggler.plot_interval = None
    runner.initializer.jiggler.n_rounds = 1
    runner.relaxers = runner.relaxers[:1]
    runner.relaxers[0].n_iterations= 10
    runner.n_generations = 100
    runner.population_size = 100
    runner.selection_size = [1,2,5,10]
kgs.profiling=False
runner.run()
#plt.plot(runner.best_cost_per_generation)

stop final relax at some point


In [ ]:
plt.plot(runner.best_cost_per_generation[:376])

In [ ]:
%debug